In [1]:
# And now we do our imports
import torch
from miniminiai.miniminiai import *
from torcheval.metrics import MulticlassAccuracy

In [2]:
from datasets import load_dataset 

# We'll use the MNIST dataset for this tutorial
dataset = load_dataset('mnist')

Reusing dataset mnist (/home/jonathan/.cache/huggingface/datasets/mnist/mnist/1.0.0/fda16c03c4ecfb13f165ba7e29cf38129ce035011519968cdaf74894ce91c9d4)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:

# Turn it into dataloaders
import torchvision.transforms.functional as TF

def transforms(b):
    b['image'] = [TF.to_tensor(o) for o in b['image']]
    return b
     
dataset = dataset.with_transform(transforms)
dls = DataLoaders.from_dd(dataset,batch_size=64)

xb, yb = next(iter(dls.train))
xb.shape, yb.shape, yb[:5]

(torch.Size([64, 1, 28, 28]), torch.Size([64]), tensor([7, 8, 2, 3, 9]))

In [4]:
from torch import nn

model = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten()
)

model(xb).shape

torch.Size([64, 10])

In [5]:
cbs = [MetricsCB(MulticlassAccuracy()), DeviceCB(), ProgressCB()]
learn = TrainLearner(model, dls, nn.CrossEntropyLoss(), lr=0.1, cbs=cbs)
learn.fit(1)

MulticlassAccuracy,loss,epoch,train
0.361,1.824,0,train
0.615,1.147,0,eval
